In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd

In [0]:
start_date = "2000-01-01"
end_date = "2099-12-31"

In [0]:
dates_df = (spark.range(0, (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days + 1)
            .withColumn("dte_sk", (lit(start_date).cast("date") + col("id").cast("int")).cast("date"))
            .drop("id")) \
            .withColumn("dte_int", date_format(col("dte_sk"), "yyyyMMdd").cast("int")) \
            .withColumn("dte_year", year(col("dte_sk"))) \
            .withColumn("dte_month", month(col("dte_sk"))) \
            .withColumn("dte_month_name", date_format(col("dte_sk"), "MMMM")) \
            .withColumn("dte_day_of_mth", dayofmonth(col("dte_sk"))) \
            .withColumn("dte_day_name", date_format(col("dte_sk"), "EEEE")) \
            .withColumn("dte_day_of_week", dayofweek(col("dte_sk"))) \
            .withColumn("dte_is_weekend", when(col("dte_day_of_week").isin(1, 7), 1).otherwise(0)) \
            .withColumn("dte_week_of_year", weekofyear(col("dte_sk"))) \
            .withColumn("dte_quarter", quarter(col("dte_sk"))) \
            .withColumn("dte_is_leap_year", when(year(col("dte_sk")) % 4 == 0, 1).otherwise(0)) \
            .orderBy("dte_sk")

In [0]:
dates_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("mta_silver.dim_date")

In [0]:
%sql
SELECT * FROM mta_silver.dim_date LIMIT 20;

In [0]:
dbutils.notebook.exit("Success")